In [1]:
from poisson_fem.mesh import PoissonFEM
import numpy as np
import scipy.sparse as sps
import scipy.sparse.linalg as lg
import time
import petsc4py
import sys
petsc4py.init(sys.argv)
from petsc4py import PETSc

In [2]:
mesh = PoissonFEM.RectangularMesh(np.ones(16)/16)
# mesh.plot()

In [3]:
def origin(x):
    return np.abs(x[0]) < np.finfo(float).eps and np.abs(x[1]) < np.finfo(float).eps
mesh.setEssentialBoundary(origin)

In [4]:
# Precomputations
Ke = mesh.compGlobStiffStencil()
K0 = PETSc.Mat().createAIJ(size=(mesh.nEq, mesh.nEq), nnz=2394)
lmbda = PETSc.Vec().createSeq(mesh.nCells)
Kvec = PETSc.Vec().createSeq(mesh.nEq**2)
x = PETSc.Vec().createSeq(mesh.nEq)
rhs = PETSc.Vec().createSeq(mesh.nEq)

# Get sparsity pattern of stiffness matrix vector
lmbda.setValues(range(mesh.nCells), np.ones(mesh.nCells))
Ke.mult(lmbda, Kvec)
nonzero_K = np.nonzero(Kvec.array)
Ktmp = sps.csr_matrix(np.reshape(Kvec.array, (mesh.nEq, mesh.nEq), order='F'))
indptr = Ktmp.indptr.copy()
print('indptr = ', indptr)
indices = Ktmp.indices.copy()
print('indice = ', indices)

# Set up solver
ksp = PETSc.KSP().create()
ksp.setType('preonly')
pc = ksp.getPC()
pc.setType('cholesky')
ksp.setFromOptions() #???

# for scipy comparison
Kes = Ke.getValuesCSR()
Kes = sps.csr_matrix((Kes[2], Kes[1], Kes[0]))

indptr =  [   0    5   11   17   23   29   35   41   47   53   59   65   71   77
   83   89   93   98  106  115  124  133  142  151  160  169  178  187
  196  205  214  223  232  238  244  253  262  271  280  289  298  307
  316  325  334  343  352  361  370  379  385  391  400  409  418  427
  436  445  454  463  472  481  490  499  508  517  526  532  538  547
  556  565  574  583  592  601  610  619  628  637  646  655  664  673
  679  685  694  703  712  721  730  739  748  757  766  775  784  793
  802  811  820  826  832  841  850  859  868  877  886  895  904  913
  922  931  940  949  958  967  973  979  988  997 1006 1015 1024 1033
 1042 1051 1060 1069 1078 1087 1096 1105 1114 1120 1126 1135 1144 1153
 1162 1171 1180 1189 1198 1207 1216 1225 1234 1243 1252 1261 1267 1273
 1282 1291 1300 1309 1318 1327 1336 1345 1354 1363 1372 1381 1390 1399
 1408 1414 1420 1429 1438 1447 1456 1465 1474 1483 1492 1501 1510 1519
 1528 1537 1546 1555 1561 1567 1576 1585 1594 1603 1612 1621 1630 1

In [5]:
N = 1e3

In [6]:
start = time.time()
for n in range(int(N)):
    # Stiffness matrix assembly, lmbda = [1, 1, ..., 1]
    lmbda.setValues(range(mesh.nCells), np.ones(mesh.nCells))
    rhs.setValues(range(mesh.nEq), np.ones(mesh.nEq))
    Ke.mult(lmbda, Kvec)
#     K = sps.csr_matrix(np.reshape(Kvec.array, (mesh.nEq, mesh.nEq), order='F'))
    K0.setValuesCSR(indptr, indices, Kvec.getValues(nonzero_K))
    K0.assemblyBegin()
    K0.assemblyEnd()

    # solving
    ksp.setOperators(K0)
    ksp.solve(rhs, x)
petsc_time = (time.time() - start)/N
print('PETSc time = ', petsc_time)

PETSc time =  0.0002727508544921875


In [7]:
start = time.time()
for n in range(int(N)):
    Kvecs = Kes @ np.ones(mesh.nCells)
#     K1 = sps.csr_matrix(np.reshape(Kvecs, (mesh.nEq, mesh.nEq), order='F'))
    K1 = sps.csr_matrix((Kvecs[nonzero_K], indices, indptr))
    x1 = lg.spsolve(K1, np.ones(mesh.nEq))
scipy_time = (time.time() - start)/N
print('scipy time = ', scipy_time)

scipy time =  0.00097763991355896


In [8]:
diff = np.linalg.norm(x.array - x1)/np.linalg.norm(x1)
print('result difference = ', diff)
print('PETSc speedup = ', scipy_time/petsc_time)

result difference =  5.032371223168723e-14
PETSc speedup =  3.5843697552447553
